In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [2]:
bank = pd.read_csv("C:\Aalesh and Mandar\Datasets\Cases\Bankruptcy\Bankruptcy.csv")

In [3]:
bank.head()

,NO,D,YR,R1,R2,R3,R4,R5,R6,R7,...,R15,R16,R17,R18,R19,R20,R21,R22,R23,R24
0,1,0,78,0.23,0.08,0.02,0.03,0.46,0.12,0.19,...,0.05,0.57,0.15,0.23,3.56,0.26,1.55,0.43,0.11,0.17
1,2,0,77,0.19,0.07,0.09,0.12,0.02,0.02,0.03,...,0.09,0.12,0.16,0.22,3.78,1.29,1.40,0.06,0.07,0.10
2,3,0,72,0.07,0.02,0.03,0.05,0.06,0.10,0.14,...,-0.03,0.02,0.02,0.04,13.29,1.61,1.43,0.03,0.05,0.07
3,4,0,80,0.07,0.03,0.04,0.04,0.04,0.06,0.06,...,-0.02,0.01,0.02,0.02,5.36,1.30,1.12,-0.06,-0.08,-0.09
4,5,0,81,0.09,0.02,0.03,0.04,0.06,0.08,0.11,...,0.02,0.07,0.10,0.14,7.74,1.48,1.41,0.03,0.04,0.06


In [4]:
x = bank.drop(["D","YR","NO"], axis=1)
y = bank["D"]

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify = y, random_state = 2022, test_size = 0.3)

In [6]:
scaler = StandardScaler()

dtc = DecisionTreeClassifier(random_state = 2022)
lda = LinearDiscriminantAnalysis()
svm = SVC(kernel="linear", probability = True,  random_state = 2022)
pipe_svm = Pipeline([("STD", scaler),("SVM",svm)])
lr = LogisticRegression()

In [7]:
dtc.fit(x_train, y_train)
y_pred = dtc.predict(x_test)
y_pred_prob = dtc.predict_proba(x_test)[:,1]
print(accuracy_score(y_test, y_pred))
roc_dtc = roc_auc_score(y_test, y_pred_prob)
print(roc_dtc)

0.8
0.8


In [8]:
svm.fit(x_train, y_train)
y_pred = svm.predict(x_test)
y_pred_prob = svm.predict_proba(x_test)[:,1]
print(accuracy_score(y_test, y_pred))
roc_svm = roc_auc_score(y_test, y_pred_prob)
print(roc_svm)

0.825
0.7875


In [10]:
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)
y_pred_prob = lr.predict_proba(x_test)[:,1]
print(accuracy_score(y_test, y_pred))
roc_lr = roc_auc_score(y_test, y_pred_prob)
print(roc_lr)

0.825
0.8025000000000001


C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
voting = VotingClassifier([("Tree", dtc),("SVM_P",pipe_svm),("LR",lr)], voting ="soft", weights = [roc_dtc, roc_svm, roc_lr])

In [12]:
voting.fit(x_train, y_train)
y_pred = voting.predict(x_test)
y_pred_prob = voting.predict_proba(x_test)[:,1]
print(accuracy_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred_prob))

0.775
0.8500000000000001


C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [14]:
voting.get_params()

{'estimators': [('Tree', DecisionTreeClassifier(random_state=2022)),
  ('SVM_P',
   Pipeline(steps=[('STD', StandardScaler()),
                   ('SVM',
                    SVC(kernel='linear', probability=True, random_state=2022))])),
  ('LR', LogisticRegression())],
 'flatten_transform': True,
 'n_jobs': None,
 'verbose': False,
 'voting': 'soft',
 'weights': [0.8, 0.7875, 0.8025000000000001],
 'Tree': DecisionTreeClassifier(random_state=2022),
 'SVM_P': Pipeline(steps=[('STD', StandardScaler()),
                 ('SVM',
                  SVC(kernel='linear', probability=True, random_state=2022))]),
 'LR': LogisticRegression(),
 'Tree__ccp_alpha': 0.0,
 'Tree__class_weight': None,
 'Tree__criterion': 'gini',
 'Tree__max_depth': None,
 'Tree__max_features': None,
 'Tree__max_leaf_nodes': None,
 'Tree__min_impurity_decrease': 0.0,
 'Tree__min_samples_leaf': 1,
 'Tree__min_samples_split': 2,
 'Tree__min_weight_fraction_leaf': 0.0,
 'Tree__random_state': 2022,
 'Tree__splitter': 'best',

In [ ]:
kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 2022)
params = {"SVM_P__SVM__C"  :  np.linspace(0.001, 5,5),
          "SVM_P__SVM__gamma" : np.linspace(0.001, 5,5),
          "LR__C":  np.linspace(0.001, 5,5),
          "TREE__max_depth" : [None,3,5],
          "TREE__min_samples_split" : [2,5,10],
          "TREE__min_samples_leaf" : [1,5]}
         
gcv = GridSearchCV(voting, param_grid = params, verbose = 3, scoring = "roc_auc", cv = kfold)
gcv.fit(x,y)

In [ ]:
print(gcv.best_params_)
print(gcv.best_score_)